In [1]:
import keras
import collections
import numpy as np
import os
import random
import re
import string

Using Theano backend.
Using gpu device 0: GeForce GTX 950 (CNMeM is disabled, cuDNN not available)


DATA PREPARATION

In [2]:
def remove_hidden(l):
    return filter(lambda s: not s[0] == '.', l)

d = "//home//xgamer//OANC-GrAF//data//written_1//journal//slate"

def get_files(d):
    files = os.listdir(d)
    files = remove_hidden(files)
    return map(lambda f: d + "//" + f, files)

dirs = get_files(d)
files = map(get_files, dirs)
files = [f for l in files for f in l]
files = filter(lambda f: f[-3:] == 'txt', files)

print len(files)

texts = []
for f in files:
    f = open(f)
    texts.append('\n'.join(f.readlines()[7:]))
    f.close()

print sum([len(text) for text in texts])

4531
27629896


In [3]:
def clean_text(text):
    text = text.strip()
    text = text.replace('\n', ' ')
    text = text.replace('--', ' - ')
    text = text.replace('\t', ' ')
    text = re.sub(r'\ \ +', r' ', text)
    text = filter(lambda char: char in string.printable, text)
    return text

texts = map(clean_text, texts)
print sum([len(text) for text in texts])

25425786


In [4]:
print texts[545]

Trustbusters Have Job Security The Dec. 14 edition of "Today's Papers" notes a Washington Post report that David Boies, the lead Justice Department attorney in the Microsoft case, has lowered the hourly fee that he is charging the government. The column asks whether this is legal, if Boies' motive was to keep his share of the work. The answer can be found in the definitive rhyming opus on antitrust law, R.W. Grant's Tom Smith and His Incredible Bread Machine : You're gouging on your prices if You charge more than the rest. But it's unfair competition If you think you can charge less! A second point that we would make To help avoid confusion Don't try to charge the same amount! That would be collusion! In short, Boies' reduction of his fee constitutes unfair competition. Of course, if he'd left his fee unchanged, that would be collusion, though in this case he'd be colluding with himself and would eventually go blind. How's that for legal clarity? - Sam Kazman Washington Bork and Mindy 

In [5]:
alphabet = collections.defaultdict(bool)

for text in texts:
    for char in text:
        alphabet[char] = True

alphabet = [char for char in alphabet if alphabet[char] == True]
alphabet.sort()

# Add a start of sequence token \t
alphabet.append('\t')
# Add an end of sequence token \n
alphabet.append('\n')

texts = map(lambda text: '\t' + text + '\n', texts)
print alphabet

char_to_index = {}
index_to_char = {}
for i in xrange(len(alphabet)):
    index_to_char[i] = alphabet[i]
    char_to_index[alphabet[i]] = i
print index_to_char
print char_to_index

[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '~', '\t', '\n']
{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: '%', 6: '&', 7: "'", 8: '(', 9: ')', 10: '*', 11: '+', 12: ',', 13: '-', 14: '.', 15: '/', 16: '0', 17: '1', 18: '2', 19: '3', 20: '4', 21: '5', 22: '6', 23: '7', 24: '8', 25: '9', 26: ':', 27: ';', 28: '<', 29: '=', 30: '>', 31: '?', 32: '@', 33: 'A', 34: 'B', 35: 'C', 36: 'D', 37: 'E', 38: 'F', 39: 'G', 40: 'H', 41: 'I', 42: 'J', 43: 'K', 44: 'L', 45: 'M', 46: 'N', 47: 'O', 48: 'P', 49: 'Q', 50: 'R', 51: 'S', 52: 'T', 53: 'U', 54: 'V', 55: 'W', 56: 'X', 57: 'Y', 58:

In [6]:
text_seq = ''.join(texts)
sample_len = 200

def sample(text, length=sample_len + 1):
    i = random.randint(0, len(text) - length - 1)
    return text[i:i+length]

def toseqs(string, alphabet=alphabet):
    assert len(string) > 1
    n = len(string) - 1
    data = np.zeros((n, len(alphabet)), dtype='uint8')
    label = np.zeros(len(alphabet), dtype='uint8')
    for i in xrange(n):
        cur = string[i]
        nxt = string[i+1]
        data[i][char_to_index[cur]] = 1
    label[char_to_index[nxt]] = 1
    return data, label

s = sample(text_seq)
print s
x,y = toseqs(s)
print x[0]
print y

def tostring(seq, alphabet=alphabet):
    string = []
    for i in xrange(seq.shape[0]):
        string.append(index_to_char[np.argmax(seq[i])])
    return ''.join(string)

print tostring(x)
print len(x), len(tostring(x))

licy of trade-plus-scolding. While demanding boldness abroad, they have opposed military action in Kosovo, Iraq, Sudan, and Afghanistan. And while falling back on Cold War rhetoric, they ignore Russia'
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
licy of trade-plus-scolding. While demanding boldness abroad, they have opposed military action in Kosovo, Iraq, Sudan, and Afghanistan. And while falling back on Cold War rhetoric, they ignore Russia
200 200


NN MODEL

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, SGD

In [8]:
model = Sequential()
print sample_len, len(alphabet)
model.add(Dense(30, input_shape=(None, len(alphabet))))
model.add(Activation('sigmoid'))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(len(alphabet)))
model.add(Activation('softmax'))
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
#rmsprop = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

200 96


In [9]:
def get_batch(size, text=text_seq):
    xs = np.zeros((size, sample_len, len(alphabet)), dtype='uint8')
    ys = np.zeros((size, len(alphabet)), dtype='uint8')
    for i in xrange(size):
        s = sample(text)
        x,y = toseqs(s)
        xs[i] = x
        ys[i] = y
    return xs, ys

valid_x, valid_y = get_batch(512)

In [ ]:
mu_acc = 0.0
    
for i in xrange(10001):
    x,y = get_batch(512)
    _,acc =  model.train_on_batch(x, y)
    mu_acc += acc
    if i % 100 == 0 and i > 0:
        print "Train accuracy: " + str(0.01*mu_acc)
        mu_acc = 0.0
        _,acc = model.test_on_batch(valid_x, valid_y)
        print "Test accuracy: " + str(acc)
        print

Train accuracy: 0.4942578125
Test accuracy: 0.505859375

Train accuracy: 0.4918359375
Test accuracy: 0.509765625

Train accuracy: 0.49271484375
Test accuracy: 0.51953125

Train accuracy: 0.4941796875
Test accuracy: 0.517578125

Train accuracy: 0.4869921875
Test accuracy: 0.5234375

Train accuracy: 0.4877734375
Test accuracy: 0.505859375

Train accuracy: 0.4902734375
Test accuracy: 0.513671875

Train accuracy: 0.4925390625
Test accuracy: 0.50390625

Train accuracy: 0.493984375
Test accuracy: 0.505859375

Train accuracy: 0.4923046875
Test accuracy: 0.513671875

Train accuracy: 0.491015625
Test accuracy: 0.517578125

Train accuracy: 0.4881640625
Test accuracy: 0.51171875

Train accuracy: 0.486484375
Test accuracy: 0.509765625

Train accuracy: 0.492109375
Test accuracy: 0.513671875

Train accuracy: 0.49416015625
Test accuracy: 0.509765625

Train accuracy: 0.49177734375
Test accuracy: 0.513671875

Train accuracy: 0.490625
Test accuracy: 0.51171875

Train accuracy: 0.4876171875
Test accuracy

In [24]:
def generate(length, seed='\t', model=model):
    seq = np.zeros((1, length + len(seed) + 1, len(alphabet)))
    for i in xrange(len(seed)):
        seq[0][i][char_to_index[seed[i]]] = 1
    string = []
    for i in xrange(length):
        c = model.predict(seq[:,:len(seed)+i+1,:])
        c = tostring(c)
        string.append(c)
        seq[0][len(seed)+i+1][char_to_index[c]] = 1
    return tostring(seq[0])

print generate(100, "\tRussia is starting to withdraw forces from Syria and its aircraft carrier group will be the first to leave, the Russian armed forces chief says.")

	Russia is starting to withdraw forces from Syria and its aircraft carrier group will be the first to leave, the Russian armed forces chief says. 	The sent the sent the sent the sent the sent the sent the sent the sent the sent the sent the sent 
